# Time‑Accurate XV‑15 rotor

This notebook walks through a **time‑accurate** setup for the XV‑15 using Flow360. The geometry consists of an isolated rotor from the XV-15 rotorcraft with a rotating zone around it. The farfield zone is cylindrical around the rotor and the rotating zone.

## 1. Create Project from Volume Mesh
- Load Python libraries and Flow360 client. If you use environment variables or tokens, initialize them here so later API calls can authenticate. XV15 data files are loaded here.

- Project is created from the volume mesh

In [1]:
import flow360 as fl
from flow360.examples import TutorialRANSXv15

TutorialRANSXv15.get_files()
project = fl.Project.from_volume_mesh(
    TutorialRANSXv15.mesh_filename,
    name="Tutorial Time-accurate RANS CFD on XV-15 from Python",
)
volume_mesh = project.volume_mesh

/home/marc/miniconda3/envs/flow360/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[13:14:22] INFO: VolumeMesh successfully submitted:                                                                
                   type   = Volume Mesh                                                                            
                   name   = Tutorial Time-accurate RANS CFD on XV-15 from Python                                   
                   id     = vm-0598497f-238d-4983-81f8-1174cfa3e960                                                
                   status = uploaded                                                                               
           

## 2. Define rotating zone
- Working in SI unit system
- The volume zone 'innerRotating' is defined with a center and axis of rotation

In [2]:
with fl.SI_unit_system:
    rotation_zone = volume_mesh["innerRotating"]
    rotation_zone.center = (0, 0, 0) * fl.u.m
    rotation_zone.axis = (0, 0, -1)

[13:15:04] INFO: using: SI unit system for unit inference.

## 3. Define SimulationParams
- The simulation parameters are defined in the python class fl.SimulationParams()

In [3]:
with fl.SI_unit_system:
    params = fl.SimulationParams(
            reference_geometry=fl.ReferenceGeometry(
                moment_center=(0, 0, 0),
                moment_length=(3.81, 3.81, 3.81),
                area=45.604,
            ),
            operating_condition=fl.AerospaceCondition(
                velocity_magnitude=5,
                alpha=-90 * fl.u.deg,
                reference_velocity_magnitude=238.14,
            ),
            time_stepping=fl.Unsteady(
                max_pseudo_steps=35,
                steps=600,
                step_size=0.5 / 600 * fl.u.s,
                CFL=fl.AdaptiveCFL(),
            ),
            outputs=[
                fl.VolumeOutput(
                    output_fields=[
                        "primitiveVars",
                        "T",
                        "Cp",
                        "Mach",
                        "qcriterion",
                        "VelocityRelative",
                    ],
                ),
                fl.SurfaceOutput(
                    surfaces=volume_mesh["*"],
                    output_fields=[
                        "primitiveVars",
                        "Cp",
                        "Cf",
                        "CfVec",
                        "yPlus",
                        "nodeForcesPerUnitArea",
                    ],
                ),
            ],
            models=[
                fl.Fluid(
                    navier_stokes_solver=fl.NavierStokesSolver(
                        absolute_tolerance=1e-9,
                        linear_solver=fl.LinearSolver(max_iterations=35),
                        limit_velocity=True,
                        limit_pressure_density=True,
                    ),
                    turbulence_model_solver=fl.SpalartAllmaras(
                        absolute_tolerance=1e-8,
                        linear_solver=fl.LinearSolver(max_iterations=25),
                        hybrid_model=fl.DetachedEddySimulation(shielding_function="DDES"),
                        rotation_correction=True,
                        equation_evaluation_frequency=1,
                    ),
                ),
                fl.Rotation(
                    volumes=rotation_zone,
                    spec=fl.AngularVelocity(600 * fl.u.rpm),
                ),
                fl.Freestream(surfaces=volume_mesh["farField/farField"]),
                fl.Wall(surfaces=volume_mesh["innerRotating/blade"]),
            ],
        )

           INFO: using: SI unit system for unit inference.

## 4. Run Case
- Run the case in the defined project

In [4]:
project.run_case(
    params=params,
    name="Tutorial Time-accurate RANS CFD on XV-15 from Python",
)

           INFO: using: SI unit system for unit inference.

[13:15:05] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = Tutorial Time-accurate RANS CFD on XV-15 from Python                                   
                   id     = case-297ba9d2-383c-4809-a8e0-9474a1295f63                                              
                   status = pending                                                                                
           

CaseMeta(name='Tutorial Time-accurate RANS CFD on XV-15 from Python', user_id='user-4141c6e3-e862-46b3-bc99-4848739f70b6', id='case-297ba9d2-383c-4809-a8e0-9474a1295f63', parent_id=None, solver_version='release-25.6', status=<Flow360Status.PENDING: 'pending'>, tags=[], created_at=None, updated_at=datetime.datetime(2025, 9, 10, 17, 15, 5, 391000, tzinfo=datetime.timezone.utc), updated_by=None, deleted=False, cloud_path_prefix=None, case_mesh_id='vm-0598497f-238d-4983-81f8-1174cfa3e960', exampleResource=False, userEmail='marc.vives@flexcompute.com', estFlexUnit=0.0, caseParentId=None, caseFinishTime=None, parentFolderId='ROOT.FLOW360', restoreAt=None, estWorkUnit=0.0, elapsedTimeInSeconds=None, metadataProcessed=False, realFlexUnit=0.0, combinedStatus='pending', running=False, caseName='Tutorial Time-accurate RANS CFD on XV-15 from Python', retryCount=None, nodeSize=0, objectRefId='case-297ba9d2-383c-4809-a8e0-9474a1295f63', success=False, refId='vm-0598497f-238d-4983-81f8-1174cfa3e960',